In [1]:
from langchain import OpenAI

import openai
import os

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_ENDPOINT  = os.environ.get("OPENAI_ENDPOINT")
OPENAI_API_VERSION  = os.environ.get("OPENAI_API_VERSION")
OPENAI_API_TYPE  = os.environ.get("OPENAI_API_TYPE")

openai.api_type = OPENAI_API_TYPE
openai.api_base = OPENAI_ENDPOINT
openai.api_version = OPENAI_API_VERSION
openai.api_key = OPENAI_API_KEY

llm = OpenAI(
    # engine = "asada001",
    # engine="aschatgpt35",
    engine="asdavinci003",    
    # model_kwargs= {
    #     "engine": "asdavinci003",
    # },
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_ENDPOINT,
)



WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [2]:
services = [
    {
        "SID": "1",
        "name": "企业查询",
        "description": "查询企业的基本信息，如成立时间、注册资本、注册地址、经营范围等",
        "params": [
            {
                "name": "vid",
                "type": "vid",
                "entity_type": "Enterprise",
                "description": "知识图谱中的企业ID",
            }
        ]
    },
    {
        "SID": "2",
        "name": "企业间投资路径查询",
        "description": "查询两个企业间的投资路径, 如A企业投资B企业, B企业投资C企业, 那么A企业和C企业间就存在投资路径",
        "params": [
            {
                "name": "from_vid",
                "type": "vid",
                "entity_type": "Enterprise",
                "description": "知识图谱中的企业ID",
            },
            {
                "name": "to_vid",
                "type": "vid",
                "entity_type": "Enterprise",
                "description": "知识图谱中的企业ID",
            },
        ]
    },
    {
        "SID": "3",
        "name": "企业实控人分析",
        "description": "查询企业的实际控制人，以及实际控制人的股权结构",
        "params": [
            {
                "name": "vid",
                "type": "vid",
                "entity_type": "Enterprise",
                "description": "知识图谱中的企业ID",
            }
        ]
    }
]

In [4]:
from custom_tools.desc_tool import DescTool

tool = DescTool.from_llm(llm=llm, services_desc=services)
tool.run("你能帮我做什么?请给我 Json 的回答，不需要原始的问题。如果你需要更多信息可以告诉我")

{'question': '\n    %INSTRUCTIONS:\n    Suppose you have some services which are described in documents. Please find the most relevant service to answer the question.\n    If more than one service is relevant, please indicate them in [sources] list.\n    \n    Please also put the 你能帮我做什么?请给我 Json 的回答，不需要原始的问题。如果你需要更多信息可以告诉我 in the [query] field.\n    \n    请用中文回答\n    \n    %QUESTION:\n    你能帮我做什么?请给我 Json 的回答，不需要原始的问题。如果你需要更多信息可以告诉我\n',
 'answer': ' 我们可以提供以下服务：企业查询、企业间投资路径查询、企业实控人分析。\n',
 'sources': '1,2,3'}

In [5]:
tool

DescTool(name='desc_retriever', description='suppose you can provide some services. When users ask you what you can do, and you can find the answer as service listed: 企业查询: 查询企业的基本信息，如成立时间、注册资本、注册地址、经营范围等\n企业间投资路径查询: 查询两个企业间的投资路径, 如A企业投资B企业, B企业投资C企业, 那么A企业和C企业间就存在投资路径\n企业实控人分析: 查询企业的实际控制人，以及实际控制人的股权结构', args_schema=<class 'custom_tools.desc_tool.DescToolInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, services_desc=[{'SID': '1', 'name': '企业查询', 'description': '查询企业的基本信息，如成立时间、注册资本、注册地址、经营范围等', 'params': [{'name': 'vid', 'type': 'vid', 'entity_type': 'Enterprise', 'description': '知识图谱中的企业ID'}]}, {'SID': '2', 'name': '企业间投资路径查询', 'description': '查询两个企业间的投资路径, 如A企业投资B企业, B企业投资C企业, 那么A企业和C企业间就存在投资路径', 'params': [{'name': 'from_vid', 'type': 'vid', 'entity_type': 'Enterprise', 'description': '知识图谱中的企业ID'}, {'name': 'to_vid', 'type': 'vid', 'entity_type': 'Enterprise', 'description': '知识图谱中的企业ID'}]}, {'SID': '3', 'name': '企业实控人分析',

In [6]:
from langchain.agents import AgentType, initialize_agent

agent = initialize_agent(
    [tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,
)

agent.apply(["你能帮我查询上海爱数的实控人吗？如果你需要更多信息，你可以问我"])



> Entering new  chain...


 我需要找到一个能够查询实控人的服务
Action: desc_retriever
Action Input: 企业实控人分析
Observation: {'question': '\n    %INSTRUCTIONS:\n    Suppose you have some services which are described in documents. Please find the most relevant service to answer the question.\n    If more than one service is relevant, please indicate them in [sources] list.\n    \n    Please also put the 企业实控人分析 in the [query] field.\n    \n    请用中文回答\n    \n    %QUESTION:\n    企业实控人分析\n', 'answer': ' 企业实控人分析可以查询企业的实际控制人，以及实际控制人的股权结构。\n', 'sources': '3'}
Thought: 我现在知道需要使用企业实控人分析服务来查询上海爱数的实控人
Action: 企业实控人分析
Action Input: 上海爱数
Observation: 企业实控人分析 is not a valid tool, try another one.
Thought: 我需要使用企业查询服务来查询上海爱数的实控人
Action: 企业查询
Action Input: 上海爱数
Observation: 企业查询 is not a valid tool, try another one.
Thought: 我需要使用desc_retriever来查询上海爱数的实控人
Action: desc_retriever
Action Input: 上海爱数
Observation: {'question': '\n    %INSTRUCTIONS:\n    Suppose you have some services which are described in documents. Please find the most relevant servic

[{'input': '你能帮我查询上海爱数的实控人吗？如果你需要更多信息，你可以问我',
  'output': '使用企业查询、企业间投资路径查询和企业实控人分析服务可以查询上海爱数的实控人。'}]

In [7]:
from __future__ import absolute_import
from custom_tools.entity_search import EntitySearchTool

entity_search = EntitySearchTool.from_llm(llm)
entity_search.run({"question": "上海爱数的实控人是谁", "entity_types": ["Enterprise"], "number": 1})

{'question': '上海爱数的实控人是谁', 'answer': {'entities': [{'enterprise': '上海爱数'}]}}

In [8]:
from langchain.agents import AgentType, initialize_agent

In [9]:
agent = initialize_agent(
    [tool, entity_search], llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True,
)

agent.apply(["你能帮我查询上海爱数的实控人吗？如果你需要更多信息，你可以问我"])



> Entering new  chain...

Action:
```
{
  "action": "EntitySearch",
  "action_input": {
    "question": "你能帮我查询上海爱数的实控人吗？",
    "entity_types": ["Enterprise", "Person"],
    "entity_number": 1
  }
}
```


Observation: {'question': '你能帮我查询上海爱数的实控人吗？', 'answer': {'entities': [{'enterprise': '上海爱数'}]}}
Thought: Get real control people
Action:
```
{
  "action": "企业实控人分析",
  "action_input": {
    "question": "你能帮我查询上海爱数的实控人吗？"
  }
}
```


Observation: 企业实控人分析 is not a valid tool, try another one.
Thought: Get real control people
Action:
```
{
  "action": "desc_retriever",
  "action_input": {
    "question": "你能帮我查询上海爱数的实控人吗？"
  }
}
```

Observation: {'question': '\n    %INSTRUCTIONS:\n    Suppose you have some services which are described in documents. Please find the most relevant service to answer the question.\n    If more than one service is relevant, please indicate them in [sources] list.\n    \n    Please also put the 你能帮我查询上海爱数的实控人吗？ in the [query] field.\n    \n    请用中文回答\n    \n

[{'input': '你能帮我查询上海爱数的实控人吗？如果你需要更多信息，你可以问我',
  'output': '我可以帮您查询上海爱数的实控人，我可以使用企业实控人分析服务来查询。'}]